# Post-hoc Screening Analysis

This notebook performs post-hoc screening of studies based on inclusion/exclusion criteria applied during abstract and full-text screening.

In [1]:
import json
import pandas as pd
from pathlib import Path
import textwrap
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch

## Load Data

In [2]:
# Load final results
with open('../nt-rev3-all_pmids/outputs/final_results.json', 'r') as f:
    data = json.load(f)

abstract_screening = data["abstract_screening_results"]
full_text_screening = data["fulltext_screening_results"]
studies = data['studies']

print(f"Total abstract screening: {len(abstract_screening)}")
print(f"Total fulltext screening: {len(full_text_screening)}")
print(f"Total studies: {len(studies)}")

Total abstract screening: 484
Total fulltext screening: 370
Total studies: 343


In [3]:
# Load criteria mapping
with open('../nt-rev3-all_pmids/outputs/criteria_mapping.json', 'r') as f:
    criteria = json.load(f)

# Extract all inclusion and exclusion criteria codes
abstract_inclusion = list(criteria['screening']['abstract']['inclusion'].keys())
abstract_exclusion = list(criteria['screening']['abstract']['exclusion'].keys())
fulltext_inclusion = list(criteria['screening']['fulltext']['inclusion'].keys())
fulltext_exclusion = list(criteria['screening']['fulltext']['exclusion'].keys())

print(f"Abstract inclusion criteria: {abstract_inclusion}")
print(f"Abstract exclusion criteria: {abstract_exclusion}")
print(f"Fulltext inclusion criteria: {fulltext_inclusion}")
print(f"Fulltext exclusion criteria: {fulltext_exclusion}")

Abstract inclusion criteria: ['I1', 'I2']
Abstract exclusion criteria: ['E1']
Fulltext inclusion criteria: ['I3', 'I4', 'I5', 'I6']
Fulltext exclusion criteria: ['E2', 'E3', 'E4', 'E5', 'E6']


## Stage 1: Exclude studies with abstract exclusion criteria

In [4]:
# Stage 1: Exclude any studies with abstract exclusion criteria applied
stage1_included = []

for study in abstract_screening:
    abstract_excl = study.get('exclusion_criteria_applied', [])
    
    if len(abstract_excl) == 0:
        stage1_included.append(study['study_id'])

print(f"Stage 1 Results:")
print(f"  Included (no abstract exclusion): {len(stage1_included)}")
print(f"  Excluded (abstract exclusion): {len(abstract_screening) - len(stage1_included)}")

Stage 1 Results:
  Included (no abstract exclusion): 449
  Excluded (abstract exclusion): 35


## Stage 2: Final inclusion based on all criteria

Studies must meet ALL inclusion criteria and have NO exclusion criteria.

In [5]:
# Stage 2: From remaining studies, include only those meeting ALL inclusion criteria
# and having NO exclusion criteria
final_included = []
final_excluded = []

for study in full_text_screening:
    # Collect all inclusion and exclusion criteria applied    
    all_applied_inclusion = set(study.get('inclusion_criteria_applied', []))
    all_applied_exclusion = study.get('exclusion_criteria_applied', [])
    
    # Check if all inclusion criteria are met
    all_inclusion_met = all(criterion in all_applied_inclusion for criterion in fulltext_inclusion)
    
    # Check if no exclusion criteria applied
    no_exclusion = len(all_applied_exclusion) == 0
    
    if all_inclusion_met and no_exclusion:
        final_included.append(study)
    else:
        final_excluded.append({
            'study': study,
            'missing_inclusion': [c for c in fulltext_inclusion if c not in all_applied_inclusion],
            'has_exclusion': all_applied_exclusion
        })

print(f"\nStage 2 Results:")
print(f"  Final included: {len(final_included)}")
print(f"  Excluded (incomplete inclusion or has exclusion): {len(final_excluded)}")


Stage 2 Results:
  Final included: 343
  Excluded (incomplete inclusion or has exclusion): 27


## Summary Statistics

In [6]:
print(f"\n=== SUMMARY ===")
print(f"\nAbstract Screening:")
print(f"  Total: {len(abstract_screening)}")
print(f"  Stage 1 included (no abstract exclusion): {len(stage1_included)}")
print(f"  Stage 1 excluded (abstract exclusion): {len(abstract_screening) - len(stage1_included)}")

print(f"\nFulltext Screening:")
print(f"  Total: {len(full_text_screening)}")
print(f"  Stage 2 final included (all criteria met): {len(final_included)}")
print(f"  Stage 2 excluded (incomplete/exclusion): {len(final_excluded)}")


=== SUMMARY ===

Abstract Screening:
  Total: 484
  Stage 1 included (no abstract exclusion): 449
  Stage 1 excluded (abstract exclusion): 35

Fulltext Screening:
  Total: 370
  Stage 2 final included (all criteria met): 343
  Stage 2 excluded (incomplete/exclusion): 27


## Analyze Exclusion Reasons

In [7]:
# Analyze abstract exclusion reasons
abstract_exclusion_counts = {}
for study in abstract_screening:
    for excl in study.get('exclusion_criteria_applied', []):
        abstract_exclusion_counts[excl] = abstract_exclusion_counts.get(excl, 0) + 1

# Analyze fulltext exclusion reasons
fulltext_missing_criteria_counts = {}
fulltext_exclusion_criteria_counts = {}

for item in final_excluded:
    for missing in item['missing_inclusion']:
        fulltext_missing_criteria_counts[missing] = fulltext_missing_criteria_counts.get(missing, 0) + 1
    
    for excl in item['has_exclusion']:
        fulltext_exclusion_criteria_counts[excl] = fulltext_exclusion_criteria_counts.get(excl, 0) + 1

print("\n=== Abstract Screening Exclusions ===")
for criterion, count in sorted(abstract_exclusion_counts.items()):
    criterion_text = criteria['screening']['abstract']['exclusion'].get(criterion, criterion)
    print(f"  {criterion}: {count} studies - {criterion_text}")

print("\n=== Fulltext Screening Exclusions ===")
print("\nMissing inclusion criteria:")
for criterion, count in sorted(fulltext_missing_criteria_counts.items()):
    criterion_text = criteria['screening']['fulltext']['inclusion'].get(criterion, criterion)
    print(f"  {criterion}: {count} studies - {criterion_text}")

print("\nExclusion criteria applied:")
for criterion, count in sorted(fulltext_exclusion_criteria_counts.items()):
    criterion_text = criteria['screening']['fulltext']['exclusion'].get(criterion, criterion)
    print(f"  {criterion}: {count} studies - {criterion_text}")


=== Abstract Screening Exclusions ===
  E1: 31 studies - systematic reviews or meta-analyses
  I1: 1 studies - I1
  I2: 3 studies - I2

=== Fulltext Screening Exclusions ===

Missing inclusion criteria:
  I3: 10 studies - Study uses functional MRI during a social-related task. Relevant constructs include (but are not limited to)- Affiliation and Attachment, Social Communication, Perception and Understanding of Self, and Perception and Understanding of Others.
  I4: 11 studies - Sample- At least one group of healthy adult participants (age 18–65). Studies that also include clinical or special groups are eligible only if results for the healthy group are reported separately.
  I5: 26 studies - Whole-brain evidence- The paper must report at least one group-level, whole-brain (voxelwise) task-evoked statistical map (e.g., main effect or within-group contrast) that is clearly identified as coming from the healthy/control group (i.e., not limited to a narrowly defined subgroup).
  I6: 24 st

## PRISMA Flow Diagram

In [8]:
# import matplotlib.pyplot as plt
# from matplotlib.patches import FancyBboxPatch, FancyArrowPatch

# # Create PRISMA flow diagram
# fig, ax = plt.subplots(figsize=(8, 10))
# ax.set_xlim(0, 10)
# ax.set_ylim(0, 22)
# ax.axis('off')

# # Helper: draw a rounded box
# def draw_box(ax, x, y, width, height, text, facecolor, fontsize=10, bold=False):
#     box = FancyBboxPatch(
#         (x, y), width, height,
#         boxstyle="round,pad=0.2",
#         facecolor=facecolor,
#         edgecolor='black',
#         linewidth=1.8,
#         mutation_aspect=0.5
#     )
#     ax.add_patch(box)
#     ax.text(
#         x + width/2, y + height/2, text,
#         ha='center', va='center',
#         fontsize=fontsize,
#         fontweight='bold' if bold else 'normal',
#         wrap=True
#     )

# # Helper: draw arrow
# def draw_arrow(ax, x1, y1, x2, y2):
#     arrow = FancyArrowPatch(
#         (x1, y1), (x2, y2),
#         arrowstyle='-|>,head_width=0.35,head_length=0.6',
#         color='black',
#         linewidth=1.8
#     )
#     ax.add_patch(arrow)

# # --- Data (replace with your variables) ---
# total_records = len(abstract_screening)
# abstract_screened = len(abstract_screening) 
# abstract_excluded = len(abstract_screening) - len(stage1_included)
# fulltext_screened = len(full_text_screening) 
# fulltext_excluded = len(final_excluded)
# final_included = len(final_included)

# # Text blocks
# abstract_exclusion_text = (
#     "Records Excluded (n=35)\n\n"
#     "E1 (n=31)\nI1 (n=1)\nI2 (n=3)"
# )

# fulltext_exclusion_text = (
#     "Missing inclusion:\n"
#     "I3 (n=11)\nI4 (n=12)\nI5 (n=12)\nI6 (n=25)\n\n"
#     "Exclusion criteria:\n"
#     "E2 (n=13)\nE3 (n=9)\nE4 (n=5)\nE5 (n=1)"
# )

# # --- Layout parameters ---
# center_x = 5
# left_x = 2.2
# right_x = 6.5
# box_width = 3.8
# right_width = 3.8
# box_height = 1.4
# v_spacing = 3  # consistent vertical gap

# # --- Draw Diagram ---
# # Top box
# top_y = 20
# draw_box(ax, left_x, top_y, 5.6, box_height,
#           f"Records Identified\n(n={total_records})",
#           facecolor="#A8E6A1", fontsize=12, bold=True)

# draw_arrow(ax, center_x, top_y, center_x, top_y - 1.2)

# # Abstract screening
# y2 = top_y - v_spacing
# draw_box(ax, left_x, y2, box_width, box_height,
#           f"Abstract Screening\n(n={abstract_screened})",
#           facecolor="#FFF7C2", fontsize=11, bold=True)
# draw_box(ax, right_x, y2, right_width, box_height,
#           abstract_exclusion_text,
#           facecolor="#FFD9D9", fontsize=9)
# draw_arrow(ax, center_x - 1, y2, center_x - 1, y2 - 1.3)

# # Full-text screening
# y3 = y2 - v_spacing
# draw_box(ax, left_x, y3, box_width, box_height,
#           f"Full-text Screening\n(n={fulltext_screened})",
#           facecolor="#FFF7C2", fontsize=11, bold=True)
# draw_box(ax, right_x, y3 - 0.3, right_width, box_height + 1.3,
#           fulltext_exclusion_text,
#           facecolor="#FFD9D9", fontsize=8.5)
# draw_arrow(ax, center_x - 1, y3, center_x - 1, y3 - 1.5)

# # Included
# y4 = y3 - v_spacing
# draw_box(ax, left_x, y4, 5.6, box_height,
#           f"Publications Included\n(n={final_included})",
#           facecolor="#A8E6A1", fontsize=12, bold=True)

# # Title
# plt.title("PRISMA Flow Diagram - Post-hoc Screening",
#           fontsize=14, fontweight='bold', pad=15)

# plt.tight_layout()
# plt.savefig("prisma_diagram_aligned.png", dpi=300, bbox_inches='tight')
# plt.show()


## Comparison with Original Status Field

In [9]:
# Get PMIDs for each group
posthoc_included_pmids = set(s['study_id'] for s in final_included)
original_included_pmids = set(s['pmid'] for s in studies if s.get('status') == 'included')

# Calculate overlaps
both_included = posthoc_included_pmids & original_included_pmids
posthoc_only = posthoc_included_pmids - original_included_pmids
original_only = original_included_pmids - posthoc_included_pmids

print(f"\n=== COMPARISON WITH ORIGINAL STATUS ===")
print(f"Original status='included': {len(original_included_pmids)}")
print(f"Post-hoc included: {len(posthoc_included_pmids)}")
print(f"\nOverlap:")
print(f"  Both included: {len(both_included)}")
print(f"  Post-hoc only: {len(posthoc_only)}")
print(f"  Original only: {len(original_only)}")
print(f"\nAgreement: {len(both_included) / len(original_included_pmids) * 100:.1f}%")


=== COMPARISON WITH ORIGINAL STATUS ===
Original status='included': 343
Post-hoc included: 343

Overlap:
  Both included: 343
  Post-hoc only: 0
  Original only: 0

Agreement: 100.0%


In [10]:
final_included

[{'study_id': '10607399',
  'decision': 'included',
  'reason': 'This is a task-based fMRI study of social-related face perception (eye gaze and identity), involving healthy adult participants (n=9, mean age 24). The paper reports group-level, whole-brain voxelwise activation results (face vs scrambled; gaze averted vs direct) with cluster thresholds (z>4) and stereotaxic coordinates, and displays figures/tables of activations. Thus it meets the meta-analysis objective and satisfies inclusion criteria for fMRI social tasks, healthy adult sample, whole-brain evidence, and acceptable reporting of whole-brain results.',
  'confidence': 0.95,
  'model_used': 'gpt-5-mini-2025-08-07',
  'screening_type': 'fulltext',
  'timestamp': '2025-11-11T14:50:51.949018',
  'inclusion_criteria_applied': ['I3', 'I4', 'I5', 'I6'],
  'exclusion_criteria_applied': []},
 {'study_id': '11496124',
  'decision': 'included',
  'reason': 'The study reports a task-based fMRI investigation of social cognition (empa

## Create DataFrame of Final Studies

In [11]:
# Create a summary dataframe for included studies
df = pd.DataFrame([{
    'pmid': s['study_id'],
    'posthoc_status': 'included',
    'inclusion': ', '.join(s.get('inclusion_criteria_applied', [])),
    'screening_reason': s.get('screening_reason', '')
} for s in final_included])

# Create dataframe for excluded studies
df_excluded = pd.DataFrame([{
    'pmid': s['study']['study_id'],
    'posthoc_status': 'excluded',
    'inclusion': ', '.join(s['study'].get('inclusion_criteria_applied', [])),
    'missing_inclusion': ', '.join(s['missing_inclusion']),
    'has_exclusion': ', '.join(s['has_exclusion']),
    'screening_reason': s['study'].get('screening_reason', '')
} for s in final_excluded])

# Combine dataframes
final_df = pd.concat([df, df_excluded], ignore_index=True)

print(f"\nTotal records in final dataframe: {len(final_df)}")
print(f"  Included: {len(df)}")
print(f"  Excluded: {len(df_excluded)}")


Total records in final dataframe: 370
  Included: 343
  Excluded: 27


In [12]:
criteria

{'screening': {'abstract': {'inclusion': {'I1': 'Functional MRI scan conducted while human participants performed a social-related task. Relevant constructs include (but are not limited to) Affiliation and Attachment, Social Communication, Perception and Understanding of Self, and Perception and Understanding of Others.',
    'I2': 'At least one group of healthy participants, age 18–65, even if other groups include participants with psychiatric or neurological disorders.'},
   'exclusion': {'E1': 'systematic reviews or meta-analyses'}},
  'fulltext': {'inclusion': {'I3': 'Study uses functional MRI during a social-related task. Relevant constructs include (but are not limited to)- Affiliation and Attachment, Social Communication, Perception and Understanding of Self, and Perception and Understanding of Others.',
    'I4': 'Sample- At least one group of healthy adult participants (age 18–65). Studies that also include clinical or special groups are eligible only if results for the health

In [13]:

# Prety print excluded studies with reasons
for item in final_excluded:
    study = item['study']
    pmid = study['study_id']
    missing_inclusion = item['missing_inclusion']
    has_exclusion = item['has_exclusion']
    
    print(f"\nPMID: {pmid}")
    if missing_inclusion:
        print(f"  Missing Inclusion Criteria: {', '.join(missing_inclusion)}")
    if has_exclusion:
        print(f"  Exclusion Criteria Applied: {', '.join(has_exclusion)}")

    # Print "reason" from screening_reason if available
    screening_reason = study.get('reason', '')
    if screening_reason:
        wrapped_reason = textwrap.fill(screening_reason, width=70, subsequent_indent='    ')
        print(f"  Screening Reason: {wrapped_reason}")


PMID: 10943684
  Missing Inclusion Criteria: I5, I6
  Exclusion Criteria Applied: E2
  Screening Reason: The study uses fMRI during a social-related face perception task with
    healthy adult participants (meets I3 and I4). However, the full
    text/abstract describes an ROI-based analysis of amygdala voxels
    and does not clearly report any group-level, whole-brain
    (voxelwise) task-evoked statistical map or whole-brain
    coordinates/figures for the healthy group. Because I5 (whole-brain
    evidence) is not satisfied and the paper appears to rely on ROI
    analyses, it violates exclusion criterion E2 (only ROI analyses
    reported). Therefore the study is excluded.

PMID: 12202103
  Missing Inclusion Criteria: I3, I4, I5, I6
  Exclusion Criteria Applied: E6
  Screening Reason: The provided full-text content contains only the References section
    and no methods, participant information, task descriptions,
    results, or whole-brain group-level statistical maps. Therefor